* Генерируем соотношения разной длины
* Генерируем слова разной длины из данного соотношения (замеряем время)
* Генерируем слова разной длины из свободной группы (замеряем время)
* Прогоняем метрики на том и на другом (замеряем время)


In [1]:
import os
import pickle
import sys
import time
from copy import deepcopy
from functools import partial
from itertools import islice
from os import environ
from pickle import dump

import matplotlib.pyplot as plt
import numpy as np
import wandb
from iteration_utilities import repeatfunc, unique_everseen
from sklearn.model_selection import train_test_split
from tokenizer import build_tokenizer
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from utils import train_collate_fn

from freegroup.derivatives import max_gamma_contains
from freegroup.sampling import CFGNormalClosureSampler, freegroup
from freegroup.sampling.helper import get_rng
from freegroup.tools import (Comm, batch_magnus_is_from_normal_closure,
                             batch_magnus_reduce_modulo_normal_closure,
                             flatten, magnus_is_from_normal_closure,
                             magnus_reduce_modulo_normal_closure, to_string)

sys.path.append('..')
from lming.metrics import batch_max_gamma_contains_ratio

* Длина соотношений [2, 50] - по пять штук
* Длина слов до 200
* Ограничение по времени / количеству сэмплов для генерации

In [2]:
seed = 0
rng = get_rng(seed)

In [51]:
fdim  = 2

max_relation_length = 20
n_relation_per_length = 3
time_limit_relation = 5 # to generate n_relation_per_length words


In [52]:
print('time to calculate relations <=', len(range(2, max_relation_length)) * time_limit_relation, 'sec')

time to calculate relations <= 90 sec


In [53]:
def sample_freegroup(sample_length, n_samples, fdim=2, time_limit=10, rng=rng):
    start = time.time()
    words = set()
    while time.time() - start < time_limit and len(words) < n_samples:
        words.add(tuple(freegroup(fdim, sample_length, rng=rng)))
    return words


In [54]:
relations = {}
for relation_length in tqdm(range(2, max_relation_length, 3)):
    relations.setdefault(relation_length, dict())
    relations_set = sample_freegroup(relation_length, n_relation_per_length)
    for relation in relations_set:
        relations[relation_length][relation] = None

  0%|          | 0/6 [00:00<?, ?it/s]

In [55]:
def sample_relation(sample_length, n_samples, sampler, time_limit=10, rng=rng):
    SIM_LEN_TIME = .1
    start = time.time()
    last_new_word_time = time.time()
    words = set()
    cur_length = len(words)
    while time.time() - start < time_limit and len(words) < n_samples:
        try:
            word = sampler(length=sample_length, rng=rng)
            words.add(tuple(word))
            if len(words) != cur_length:
                last_new_word_time = time.time()
                cur_length = len(words)
        except ValueError:
            pass
    
        if len(words) == cur_length and time.time() - last_new_word_time > SIM_LEN_TIME:
            return words
        
    return words


In [56]:
max_word_length = 50
n_words_per_length = 50
time_limit_word = 1 # to generate n_words_per_length words

In [57]:
time_per_relation = time_limit_word * max_word_length
print('time to generate words per relation <=', time_per_relation, 'sec')
print('time to generate words overall <=', time_per_relation * len(relations) * n_relation_per_length, 'sec')

time to generate words per relation <= 50 sec
time to generate words overall <= 900 sec


In [58]:
# samplers = {key: [] for key in relations}
# samplers_time = {key: [] for key in relations} 

# for relation_length, relation_dict in tqdm(relations.items()):
#     for relation in relation_dict:
#         # Create sampler
#         sampler_start = time.time()
#         sampler = CFGNormalClosureSampler.build(closure=list(relation),
#                                                 fdim=fdim,
#                                                 max_length=max_word_length)
#         sampler_end = time.time()
#         samplers_time[relation_length].append(sampler_end - sampler_start)
#         samplers[relation_length].append(sampler)

ОТДЕЛЬНО ПОСЧИТАТЬА ДЛЯ СЛОВ ИЗ СВОИХ ЗАМЫКАНИЙ И ДЛЯ СВОБОДНОЙ ГРУППЫ

### Для замыканий

In [59]:
# for relation_length, relation_dict in tqdm(relations.items()):
#     for relation, sampler in zip(relation_dict.keys(), samplers[relation_length]):
#         words = {}
#         # Здесь надо насэмплить слова
#         for word_length in range(2, max_word_length):
#             words[word_length] = sample_relation(word_length, n_words_per_length, sampler, time_limit=time_limit_word)



#         cr_table = np.empty((max_word_length)) * np.nan
#         mg_table = np.empty((max_word_length)) * np.nan

#         # CALCULATE IS_FROM_NORMAL_CLOSURE
#         for word_length, word_set in words.items():
#             if word_set:
#                 start = time.time()
#                 for word in word_set:
#                     tmp = magnus_is_from_normal_closure(word, relation)
#                 end = time.time()

                
#                 cr_table[word_length] = (end - start) / len(word_set)

#         # CALCULATE MAX_GAMMA_CONTAINS
#         for word_length, word_set in words.items():
#             if word_set:
#                 start = time.time()
#                 for word in word_set:
#                     tmp = max_gamma_contains(list(word), fdim=fdim)
#                 end = time.time()

                
#                 mg_table[word_length] = (end - start) / len(word_set)        

#         # Потом посчитать метрики в среднем по длине слова    

#         relations[relation_length][relation] = np.array([cr_table, mg_table])
#         break
#     break
        

        

#         # relations[relation_length][relation] = столбик размером с количество длин слов

### Для свободной группы

In [ ]:
for relation_length, relation_dict in tqdm(relations.items()):
    for relation in tqdm(relation_dict.keys(), leave=False):
        words = {}
        # Здесь надо насэмплить слова
        for word_length in range(2, max_word_length, 3):
            words[word_length] = sample_freegroup(word_length, n_words_per_length, time_limit=time_limit_word)



        cr_table = np.empty((max_word_length)) * np.nan
        mg_table = np.empty((max_word_length)) * np.nan

        # CALCULATE IS_FROM_NORMAL_CLOSURE
        for word_length, word_set in words.items():
            if word_set:
                start = time.time()
                for word in word_set:
                    tmp = magnus_is_from_normal_closure(word, relation)
                end = time.time()

                
                cr_table[word_length] = (end - start) / len(word_set)

        # CALCULATE MAX_GAMMA_CONTAINS
        for word_length, word_set in words.items():
            if word_set:
                start = time.time()
                for word in word_set:
                    tmp = max_gamma_contains(list(word), fdim=fdim)
                end = time.time()

                
                mg_table[word_length] = (end - start) / len(word_set)        

        # Потом посчитать метрики в среднем по длине слова    

        relations[relation_length][relation] = np.array([cr_table, mg_table])
        

        

        # relations[relation_length][relation] = столбик размером с количество длин слов

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [107]:
magnus_is_from_normal_closure(word, relation)

KeyboardInterrupt: 

In [97]:
print(word)

(1, 2, -1, 2, 1, -2, 1, 2, -1, 2, 2, 2, 2, -1, -2, 1, 2, -1, -2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, -1, 2, 2, 2, 2, 1, 2, 1, -2, -2, 1, 2, 2)


In [98]:
relation

(2, 2, 2, -1, -2, -1, -1, -1, -1, -1, 2)

In [ ]:
relations[2][(1,-2)]

AttributeError: 'numpy.ndarray' object has no attribute 'nanmean'

In [106]:
relations[2][(1, -2)][0]

array([       nan,        nan, 0.0002627 ,        nan,        nan,
       0.00038651,        nan,        nan, 0.00051949,        nan,
              nan, 0.00065054,        nan,        nan, 0.00078794,
              nan,        nan, 0.00092748,        nan,        nan,
       0.00106255,        nan,        nan, 0.00118923,        nan,
              nan, 0.0013168 ,        nan,        nan, 0.00146978,
              nan,        nan, 0.0016012 ,        nan,        nan,
       0.00173427,        nan,        nan, 0.00185146,        nan,
              nan, 0.00197187,        nan,        nan, 0.00215027,
              nan,        nan, 0.00225279,        nan,        nan])

In [104]:
relations[11][(2,
  1,
  1,
  1,
  2,
  -1,
  -2,
  -1,
  2,
  -1,
  -2)][0]

array([       nan,        nan, 0.00095208,        nan,        nan,
       0.00153477,        nan,        nan, 0.00332066,        nan,
              nan, 0.0048939 ,        nan,        nan, 0.00638343,
              nan,        nan, 0.00875181,        nan,        nan,
       0.01582205,        nan,        nan, 0.01502557,        nan,
              nan, 0.01514737,        nan,        nan, 0.02404424,
              nan,        nan, 0.02713127,        nan,        nan,
       0.03287436,        nan,        nan, 0.03613797,        nan,
              nan, 0.04956366,        nan,        nan, 0.04982257,
              nan,        nan, 0.05512855,        nan,        nan])

In [91]:
relations[11].keys()

dict_keys([(2, 1, 1, 1, 2, -1, -2, -1, 2, -1, -2), (2, 2, 2, -1, -2, -1, -1, -1, -1, -1, 2), (2, 1, 1, 2, -1, -2, -1, -1, -2, 1, 2)])

In [65]:
relations.keys()

dict_keys([2, 5, 8, 11, 14, 17])

In [66]:
word_length

47

In [67]:
word

(1,
 2,
 -1,
 2,
 1,
 -2,
 1,
 2,
 -1,
 2,
 2,
 2,
 2,
 -1,
 -2,
 1,
 2,
 -1,
 -2,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 -1,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 -2,
 -2,
 1,
 2,
 2)

In [24]:
import joblib
joblib.dump(relations, 'relations_time_experiment.pkl')

['relations_time_experiment.pkl']

In [ ]:
relations.

In [4]:

L = 100
fdim =2 
length = 2

In [43]:
start = time.time()
words = set()
while time.time() - start < time_limit and len(words) < n_samples:
    words.add(to_string(freegroup(fdim, length, rng=rng)))


  0%|          | 0/100 [00:00<?, ?it/s]

In [25]:
{to_string(freegroup(2, 2)), to_string(freegroup(2, 2))}

{'2 -1'}

In [17]:
def fn():
    word = freegroup(fdim, length, rng=rng)
    return word

iterator = repeatfunc(fn)
iterator = unique_everseen(iterator)
# iterator = islice(iterator, 100)
words = []
for el in iterator:
    words.append(el)

# return list(tqdm(iterator, total=int(n_samples)))

In [43]:
def sample_freegroup(fdim=2, length=2, n_samples=int(1e3), max_time=100, rng=rng): 
    # функция асэмплит слов с определенными fdim и length
    # label = 'f'
    start = time.start()
    def fn():
        word = freegroup(fdim, length, rng=rng)
        return word
            # {
            # "word_str" : to_string(word),
            # "label": label,
            # }
    iterator = repeatfunc(fn)
    iterator = unique_everseen(iterator)
    iterator = islice(iterator, n_samples)

    return list(tqdm(iterator, total=int(n_samples)))